The first thing we want to do is read in the data, explore, take a sample and create new variables. 

In [5]:
# initialisation and read the data
import os 
import pandas as pd
import random
from sklearn.decomposition import PCA
os.chdir('/Users/jpmallette/Downloads/')

destinations = pd.read_csv("destinations.csv")
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")

In [10]:
# create some variables useful for downsampling 
train["date_time"] = pd.to_datetime(train["date_time"])
train["year"] = train["date_time"].dt.year
train["month"] = train["date_time"].dt.month

In [8]:
# quick exploration
train.shape

(37670293, 24)

In [20]:
# test, 
# sample data preparation
def draw_sample(df,row):

    unique_users = train.user_id.unique()

    sel_user_ids = [unique_users[i] for i in sorted(random.sample(range(len(unique_users)), row)) ]
    sel_train = train[train.user_id.isin(sel_user_ids)]

    sample_train = sel_train[((sel_train.year == 2013) | ((sel_train.year == 2014) & (sel_train.month < 8)))]
    sample_test = sel_train[((sel_train.year == 2014) & (sel_train.month >= 8))]

    # remove click events
    sample_test = sample_test[sample_test.is_booking == True]
    return sample_train,sample_test

# draw the sample
sample_train, sample_test = draw_sample(train,10000)

In [ ]:
Create new variables 

In [ ]:
# temp asignment
# sample_train.loc[:,'srch_children_cnt'].iloc[0]
# bad and slow coding : help would be appreciated

# number of people in trip categorisation
sample_train["nb_person"] =  sample_train['srch_adults_cnt'] + sample_train['srch_children_cnt']

# family variables
df['family'] = df['site_name']

for row in range(len(sample_train)):    
    if df['srch_adults_cnt'].iloc[row] == 1 & df['srch_children_cnt'].iloc[row] == 0:
        df['family'].iloc[row] = 'single'
    elif df['srch_adults_cnt'].iloc[row] == 1 & df['srch_children_cnt'].iloc[row] == 1:
        df['family'].iloc[row] = 'monoparental'
    elif df['srch_adults_cnt'].iloc[row] == 2 & df['srch_children_cnt'].iloc[row] == 0:
        df['family'].iloc[row] = 'couple'
    elif df['srch_adults_cnt'].iloc[row] == 2 & (df['srch_children_cnt'].iloc[row] == 1 | 
                                                 df['srch_children_cnt'].iloc[row] == 2 |
                                                 df['srch_children_cnt'].iloc[row] == 3 | 
                                                 df['srch_children_cnt'].iloc[row] == 4):
        df['family'].iloc[row] = 'family'
    else:
        df['family'].iloc[row] = 'group'
    print(row)

In [ ]:
# number of person
def peron_features(df):

    # number of people in trip categorisation
    df["nb_person"] =  df['srch_adults_cnt'] + df['srch_children_cnt']

    # number of people in trip categorisation
    df["family"] = 'temp'
    single = df['nb_person'] == 1 
    monoparental = df['srch_adults_cnt'] == 1 & df['srch_children_cnt'] == 1
    couple = df['srch_adults_cnt'] == 2 & df['srch_children_cnt'] == 0
    family = df['srch_adults_cnt'] == 2 & (df['srch_children_cnt'] == 1 | df['srch_children_cnt'] == 2 
                                               | df['srch_children_cnt'] == 3 | df['srch_children_cnt'] == 4)
    group = df["nb_person"] > 6

    df.loc[single, 'family'] = "single"
    df.loc[monoparental, 'family'] = "monoparental"
    df.loc[couple, 'family'] = "couple"
    df.loc[family, 'family'] = "family"                                        
    df.loc[group, 'family'] = "group"
                                           
    return df["nb_person"], df["family"]
    
# duration of stay in days

def calc_fast_features(df):
    df["date_time"] = pd.to_datetime(df["date_time"])
    df["srch_ci"] = pd.to_datetime(df["srch_ci"], format='%Y-%m-%d', errors="coerce")
    df["srch_co"] = pd.to_datetime(df["srch_co"], format='%Y-%m-%d', errors="coerce")
    
    props = {}
    for prop in ["month", "day", "hour", "minute", "dayofweek", "quarter"]:
        props[prop] = getattr(df["date_time"].dt, prop)
    
    carryover = [p for p in df.columns if p not in ["date_time", "srch_ci", "srch_co"]]
    for prop in carryover:
        props[prop] = df[prop]
    
    date_props = ["month", "day", "dayofweek", "quarter"]
    for prop in date_props:
        props["ci_{0}".format(prop)] = getattr(df["srch_ci"].dt, prop)
        props["co_{0}".format(prop)] = getattr(df["srch_co"].dt, prop)
    props["stay_span"] = (df["srch_co"] - df["srch_ci"]).astype('timedelta64[h]')
        
    ret = pd.DataFrame(props)
    
    ret = ret.join(dest_small, on="srch_destination_id", how='left', rsuffix="dest")
    ret = ret.drop("srch_destination_iddest", axis=1)
    return ret

df = calc_fast_features(t1)
df.fillna(-1, inplace=True)
                                        
# PCA 

pca = PCA(n_components=3)
dest_small = pca.fit_transform(destinations[["d{0}".format(i + 1) for i in range(149)]])
dest_small = pd.DataFrame(dest_small)
dest_small["srch_destination_id"] = destinations["srch_destination_id"]

In [3]:
# evaluation for algorithm

import ml_metrics as metrics
target = [[l] for l in t2["hotel_cluster"]]
metrics.mapk(target, predictions, k=5)

    100% |████████████████████████████████| 1.2MB 545kB/s 
  Found existing installation: pip 8.1.1
    Uninstalling pip-8.1.1:
Exception:
Traceback (most recent call last):
  File "/Users/jpmallette/anaconda/lib/python2.7/site-packages/pip/basecommand.py", line 209, in main
    status = self.run(options, args)
  File "/Users/jpmallette/anaconda/lib/python2.7/site-packages/pip/commands/install.py", line 317, in run
    prefix=options.prefix_path,
  File "/Users/jpmallette/anaconda/lib/python2.7/site-packages/pip/req/req_set.py", line 726, in install
    requirement.uninstall(auto_confirm=True)
  File "/Users/jpmallette/anaconda/lib/python2.7/site-packages/pip/req/req_install.py", line 746, in uninstall
    paths_to_remove.remove(auto_confirm)
  File "/Users/jpmallette/anaconda/lib/python2.7/site-packages/pip/req/req_uninstall.py", line 115, in remove
    renames(path, new_path)
  File "/Users/jpmallette/anaconda/lib/python2.7/site-packages/pip/utils/__init__.py", line 267, in renames
 